In [1]:
!pip install transformers datasets sqlparse torch sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

In [5]:
import json
import pandas as pd

# Load training data
with open("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_spider.json") as f:
    train_data = json.load(f)

# Load dev (validation) data
with open("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/dev.json") as f:
    dev_data = json.load(f)

# Load database schemas (for schema linking)
with open("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/tables.json") as f:
    tables_data = json.load(f)

# Create a schema dictionary {db_id: schema_info}
db_schemas = {db["db_id"]: db for db in tables_data}

In [7]:
# Check the structure of the first entry in tables.json
print(db_schemas[list(db_schemas.keys())[0]])

{'column_names': [[-1, '*'], [0, 'perpetrator id'], [0, 'people id'], [0, 'date'], [0, 'year'], [0, 'location'], [0, 'country'], [0, 'killed'], [0, 'injured'], [1, 'people id'], [1, 'name'], [1, 'height'], [1, 'weight'], [1, 'home town']], 'column_names_original': [[-1, '*'], [0, 'Perpetrator_ID'], [0, 'People_ID'], [0, 'Date'], [0, 'Year'], [0, 'Location'], [0, 'Country'], [0, 'Killed'], [0, 'Injured'], [1, 'People_ID'], [1, 'Name'], [1, 'Height'], [1, 'Weight'], [1, 'Home Town']], 'column_types': ['text', 'number', 'number', 'text', 'number', 'text', 'text', 'number', 'number', 'number', 'text', 'number', 'number', 'text'], 'db_id': 'perpetrator', 'foreign_keys': [[2, 9]], 'primary_keys': [1, 9], 'table_names': ['perpetrator', 'people'], 'table_names_original': ['perpetrator', 'people']}


In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')


2025-04-27 08:56:35.711562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745744195.955882      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745744196.019449      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
import json
from pathlib import Path

# Set your paths (change this if needed)
data_dir = Path('/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider')

# Load Spider files
with open(data_dir / 'train_spider.json', 'r') as f:
    train_data = json.load(f)

with open(data_dir / 'tables.json', 'r') as f:
    tables = json.load(f)

# Build a mapping: db_id -> schema string
def build_schema(tables):
    schemas = {}
    for table in tables:
        db_id = table['db_id']
        schema = []
        for table_name in table['table_names_original']:
            schema.append(f"table: {table_name}")
        for column in table['column_names_original']:
            table_id, column_name = column
            if table_id >= 0:
                table_name = table['table_names_original'][table_id]
            else:
                table_name = "none"
            schema.append(f"column: {column_name} (table: {table_name})")
        schemas[db_id] = " | ".join(schema)
    return schemas


schemas = build_schema(tables)

# Prepare (input, output) pairs
train_pairs = []
for item in train_data:
    question = item['question']
    db_id = item['db_id']
    sql = item['query']

    schema = schemas[db_id]

    input_text = f"question: {question} schema: {schema}"
    output_text = sql

    train_pairs.append({'input': input_text, 'output': output_text})

print(f"Loaded {len(train_pairs)} training examples!")


Loaded 7000 training examples!


In [12]:
# ===============================
# 1. Install + Import Libraries
# ===============================
!pip install transformers datasets

import json
from pathlib import Path
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [13]:
# ===============================
# 2. Load Spider Dataset
# ===============================
data_dir = Path('/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider')

with open(data_dir / 'train_spider.json', 'r') as f:
    train_data = json.load(f)

with open(data_dir / 'dev.json', 'r') as f:
    dev_data = json.load(f)

with open(data_dir / 'tables.json', 'r') as f:
    tables = json.load(f)

In [14]:
# ===============================
# 3. Preprocessing: Flatten the Schema
# ===============================
def build_schema(tables):
    schemas = {}
    for table in tables:
        db_id = table['db_id']
        schema = []
        for table_name in table['table_names_original']:
            schema.append(f"table: {table_name}")
        for column in table['column_names_original']:
            table_id, column_name = column
            if table_id >= 0:
                table_name = table['table_names_original'][table_id]
            else:
                table_name = "none"
            schema.append(f"column: {column_name} (table: {table_name})")
        schemas[db_id] = " | ".join(schema)
    return schemas

schemas = build_schema(tables)

def prepare_examples(data, schemas):
    examples = []
    for item in data:
        question = item['question']
        db_id = item['db_id']
        sql = item['query']

        schema = schemas.get(db_id, "")

        input_text = f"question: {question} schema: {schema}"
        output_text = sql

        examples.append({'input': input_text, 'output': output_text})
    return examples

train_examples = prepare_examples(train_data, schemas)
dev_examples = prepare_examples(dev_data, schemas)

print(f"Loaded {len(train_examples)} training examples.")
print(f"Loaded {len(dev_examples)} validation examples.")

Loaded 7000 training examples.
Loaded 1034 validation examples.


In [33]:
# ===============================
# 4. Load Tokenizer and Model
# ===============================
model_name = "t5-base"  # You can change to 't5-base' or 't5-large'

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [34]:
# ===============================
# 5. Tokenize the Dataset
# ===============================
def preprocess_function(examples):
    # Tokenize inputs (questions + schema)
    model_inputs = tokenizer(
        examples['input'],
        max_length=512,
        truncation=True,
        padding='max_length'  # <--- add padding
    )

    # Tokenize targets (SQL queries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['output'],
            max_length=512,
            truncation=True,
            padding='max_length'  # <--- add padding
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


train_dataset = Dataset.from_list(train_examples).map(preprocess_function, batched=True)
dev_dataset = Dataset.from_list(dev_examples).map(preprocess_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [35]:
# ===============================
# 6. Setup Training Arguments
# ===============================
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    #evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    predict_with_generate=True,
    fp16=True,  # Use mixed precision if GPU supports
    report_to="none",  # Disable Wandb logging
)

In [36]:
# ===============================
# 7. Create Trainer
# ===============================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
)

# ===============================
# 8. Start Fine-Tuning
# ===============================
trainer.train()


/tmp/ipykernel_31/2702171312.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,1.116400
200,0.088400
300,0.067400
400,0.053800
500,0.048000
600,0.044800
700,0.041400
800,0.037100
900,0.036400
1000,0.036300


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1750, training_loss=0.10316231734412057, metrics={'train_runtime': 2624.3494, 'train_samples_per_second': 5.335, 'train_steps_per_second': 0.667, 'total_flos': 8525410467840000.0, 'train_loss': 0.10316231734412057, 'epoch': 2.0})

In [37]:
trainer.save_model("./t5_spider_finetuned")


In [38]:
def predict_sql(question, schema_text):
    input_text = f"question: {question} schema: {schema_text}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)  # <-- MOVE TO DEVICE
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example
test_question = "What are the names of all singers?"
test_schema = schemas['concert_singer']  # Choose db_id
print(predict_sql(test_question, test_schema))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


SELECT Name FROM singer ORDER BY Song_Name DESC LIMIT 1


In [39]:
import json
import sqlite3
from tqdm import tqdm
import os
import sqlparse

# 1. Load dev set
with open("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/dev.json", "r") as f:
    dev_data = json.load(f)

# 2. Execution checking helper
def execute_sql(db_path, sql_query):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return None

# 3. Evaluation loop
def evaluate(model, tokenizer, schemas, dev_data, db_folder="/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database"):
    exact_match = 0
    execution_match = 0
    total = 0

    correct_examples = []
    wrong_examples = []

    for example in tqdm(dev_data):
        question = example['question']
        db_id = example['db_id']
        gold_sql = example['query']

        # Input text
        schema_text = schemas[db_id]
        input_text = f"question: {question} schema: {schema_text}"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)

        # Predict
        outputs = model.generate(**inputs, max_length=512)
        pred_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Normalize SQLs
        pred_norm = sqlparse.format(pred_sql, keyword_case="lower", strip_comments=True).strip()
        gold_norm = sqlparse.format(gold_sql, keyword_case="lower", strip_comments=True).strip()

        # 1. Exact Match
        if pred_norm == gold_norm:
            exact_match += 1
            correct_examples.append((question, pred_sql, gold_sql))
        else:
            wrong_examples.append((question, pred_sql, gold_sql))

        # 2. Execution Match
        db_path = os.path.join(db_folder, db_id, db_id + ".sqlite")
        if os.path.exists(db_path):
            gold_result = execute_sql(db_path, gold_sql)
            pred_result = execute_sql(db_path, pred_sql)

            if gold_result == pred_result and gold_result is not None:
                execution_match += 1

        total += 1

    exact_match_acc = exact_match / total
    execution_acc = execution_match / total

    return exact_match_acc, execution_acc, correct_examples, wrong_examples

# 4. Run
exact_match_acc, execution_acc, correct_examples, wrong_examples = evaluate(model, tokenizer, schemas, dev_data)

print(f"Exact Match Accuracy: {exact_match_acc * 100:.2f}%")
print(f"Execution Match Accuracy: {execution_acc * 100:.2f}%")

# 5. Show Examples
print("\n✅ Correct Predictions (Sample 5):")
for q, pred, gold in correct_examples[:5]:
    print(f"\nQuestion: {q}\nPredicted SQL: {pred}\nGold SQL: {gold}\n")

print("\n❌ Wrong Predictions (Sample 5):")
for q, pred, gold in wrong_examples[:5]:
    print(f"\nQuestion: {q}\nPredicted SQL: {pred}\nGold SQL: {gold}\n")


100%|██████████| 1034/1034 [14:12<00:00,  1.21it/s] 

Exact Match Accuracy: 3.19%
Execution Match Accuracy: 15.57%

✅ Correct Predictions (Sample 5):

Question: What is the total number of singers?
Predicted SQL: SELECT count(*) FROM singer
Gold SQL: SELECT count(*) FROM singer


Question: What are the names of the stadiums without any concerts?
Predicted SQL: SELECT name FROM stadium WHERE stadium_id NOT IN (SELECT stadium_id FROM concert)
Gold SQL: SELECT name FROM stadium WHERE stadium_id NOT IN (SELECT stadium_id FROM concert)


Question: Find the number of distinct type of pets.
Predicted SQL: SELECT count(DISTINCT pettype) FROM pets
Gold SQL: SELECT count(DISTINCT pettype) FROM pets


Question: How many employees are there?
Predicted SQL: SELECT count(*) FROM employee
Gold SQL: SELECT count(*) FROM employee


Question: Count the number of employees
Predicted SQL: SELECT count(*) FROM employee
Gold SQL: SELECT count(*) FROM employee


❌ Wrong Predictions (Sample 5):

Question: How many singers do we have?
Predicted SQL: SELECT count(